.. _errors

Error calculations
==================

FEMpy provides the ability to compute the $L^\infty$ and $L^2$ norm errors as well as the $H^1$ semi-norm error. 

This tutorial was made with the following version of FEMpy:	

In [1]:
import FEMpy
FEMpy.__version__

'1.0'

Let us examine the error of

$$-\frac{{\rm d}}{{\rm d} x} \left( e^x \frac{{\rm d}}{{\rm d} x} u(x)\right) = -e^x [\cos x -2\sin x - x \cos x - x \sin x]; x \in [0, 1]$$
$$u(0) = 0, u'(1) = \cos(1) - \sin(1)$$

as we vary the mesh step size $h$.


In [2]:
import numpy as np


def coefficient_function(x): 
    return  np.exp(x)


def source_function(x):
    return -np.exp(x) * (np.cos(x) - 2*np.sin(x) - x*np.cos(x) - x*np.sin(x))


def dirichlet_function(x):
    if x == 0:
        return 0

    
def neumann_function(x):
    if x == 1:
        return np.cos(1) - np.sin(1)

We will need the analytical solution to our problem for the $L^\infty$ and $L^2$ norm errors and the derivative of the solution for the $H^1$ semi-norm.

In [3]:
def analytical_sol(x):
    return x * np.cos(x)


def dx_analytical_sol(x):
    return np.cos(x) - x * np.sin(x)

We will vary our mesh size for $h \in  \left\{ \frac{1}{4}, \frac{1}{8}, \frac{1}{16}, \frac{1}{32}, \frac{1}{128}, \frac{1}{256} \right\}.$

In [4]:
h_list = [1/(2**n) for n in np.arange(2, 9)]

For our case we will use quadratic finite elements

In [5]:
basis = FEMpy.IntervalBasis1D('quadratic')

Now we can iterate through our mesh sizes and store our errors.

In [6]:
L_inf_err = []
L2_err = []
H1_err = []
for h in h_list:
    mesh = FEMpy.Interval1D(0, 1, h, 'quadratic')
    bcs = FEMpy.BoundaryConditions(mesh, ('dirichlet', 'neumann'), dirichlet_fun=dirichlet_function, neumann_fun=neumann_function, coeff_fun=coefficient_function)
    
    poisson_eq = FEMpy.Poisson1D(mesh, fe_trial_basis=basis, fe_test_basis=basis, boundary_conditions=bcs)
    poisson_eq.solve(coefficient_function, source_function)
    
    L_inf_err.append(poisson_eq.l_inf_error(analytical_sol))
    L2_err.append(poisson_eq.l2_error(analytical_sol))
    H1_err.append(poisson_eq.h1_seminorm_error(dx_analytical_sol))

To display our results we can use a [pandas](https://pandas.pydata.org/)  dataframe or an [astropy](http://www.astropy.org/) table.

In [7]:
from astropy.table import Table

error_table = Table([h_list, L_inf_err, L2_err, H1_err], names=['h', 'L_inf Norm Error', 'L_2 Norm Error', 'H_1 Semi-norm Error'],)
error_table['h'].format = '.4f'; error_table['L_inf Norm Error'].format = '.4e'; error_table['L_2 Norm Error'].format = '.4e'; error_table['H_1 Semi-norm Error'].format = '.4e'
error_table

<i>Table length=7</i>
<table id="table133885012620344" class="table-striped table-bordered table-condensed">
<thead><tr><th>h</th><th>L_inf Norm Error</th><th>L_2 Norm Error</th><th>H_1 Semi-norm Error</th></tr></thead>
<thead><tr><th>float64</th><th>float64</th><th>float64</th><th>float64</th></tr></thead>
<tr><td>0.2500</td><td>3.3279e-04</td><td>2.1050e-04</td><td>5.4213e-03</td></tr>
<tr><td>0.1250</td><td>3.9288e-05</td><td>2.6147e-05</td><td>1.3534e-03</td></tr>
<tr><td>0.0625</td><td>4.7533e-06</td><td>3.2632e-06</td><td>3.3823e-04</td></tr>
<tr><td>0.0312</td><td>5.8395e-07</td><td>4.0774e-07</td><td>8.4550e-05</td></tr>
<tr><td>0.0156</td><td>7.2344e-08</td><td>5.0962e-08</td><td>2.1137e-05</td></tr>
<tr><td>0.0078</td><td>9.0022e-09</td><td>6.3701e-09</td><td>5.2842e-06</td></tr>
<tr><td>0.0039</td><td>1.1227e-09</td><td>7.9626e-10</td><td>1.3211e-06</td></tr>
</table>